In [31]:
import chess.pgn
import pandas as pd
from tqdm import tqdm
import io
import time

In [32]:
def get_eval(game,n = None, t = None, threads =8):
    # Note - I would like to make a class which contains all important metrics for my neural net
    board = game.board()
    move_number = 0
    white = []
    black = []
    centipawns = []
    engine = chess.engine.SimpleEngine.popen_uci("stockfish_15_win_x64_avx2\\stockfish_15_x64_avx2.exe")
    engine.configure({"Threads": threads}) # or even higher

    for move in game.mainline_moves():
        board.push(move)
        if move_number == 0:
            info = engine.analyse(board, chess.engine.Limit(depth = n,time=t))
            prev_centipawn = info["score"].white().score()
        if move_number > 0:
            info = engine.analyse(board, chess.engine.Limit(depth = n,time = t))
            centipawn = info["score"].white().score()
            if centipawn is None:
                centipawn = 0
            if move_number % 2 == 0:
                white.append(centipawn - prev_centipawn)
            else:
                black.append(- (centipawn - prev_centipawn))
            prev_centipawn = centipawn
            centipawns.append(centipawn)
        move_number +=1
    engine.quit()
    return white, black, centipawns

In [33]:
class Eval:
    def __init__(self, game, n, time_limit,threads):
        self.game = game
        self.white_eval,self.black_eval,self.centipawns  = get_eval(self.game, n, time_limit,threads)

In [34]:
def to_eval(x,n, t,threads):
    eval = Eval(x,n,t,threads )
    return eval

In [35]:
def pgn_to_csv(player, time_limit,threads):
    i = 0
    file2 = open('pgns/' +player + '.pgn',encoding="utf-8",errors = "ignore")

    #TODO - change this to a more standard while loop
    game = chess.pgn.read_game(file2)
    df = pd.read_csv('blitz/' + player +'.csv')
    df = df[["White", "Black", "WhiteELO", "BlackELO", "Date","Event", "Result", "Opening", "Rounds", "Game", "Eval", "EventType"]]
    j = len(df.index)
    while isinstance(game,chess.pgn.Game):
        white = []
        black = []
        white_elo = []
        black_elo = []
        date = []
        event = []
        result = []
        wdiff = []
        bdiff = []
        rounds = []
        opening = []
        games = []
        evals = []
        i +=1
        if i <= j:
            game = chess.pgn.read_game(file2)
            continue
        start = time.time()
        print(i)
        if "White" in game.headers:
            white.append(game.headers["White"])
        else:
            white.append('*')
        if "Round" in game.headers:
            rounds.append(game.headers["Round"])
        else:
            white.append('*')
        if "Black" in game.headers:
            black.append(game.headers["Black"])
        else:
            black.append('*')
        if "WhiteElo" in game.headers:
            white_elo.append(game.headers["WhiteElo"])
        else:
            white_elo.append('*')
        if "BlackElo" in game.headers:
            black_elo.append(game.headers["BlackElo"])
        else:
            black_elo.append('*')
        if "Date" in game.headers:
            date.append(game.headers["Date"])
        else:
            date.append('*')
        if "EventType" in game.headers:
            event.append(game.headers["EventType"])
            if 'blitz' in game.headers["EventType"]:
                evals.append(to_eval(game ,20, time_limit,threads).centipawns)
            else:
                evals.append('')
        else:
            event.append('*')
            evals.append('')
        if "ECO" in game.headers:
            opening.append(game.headers["ECO"])
        else:
            opening.append('*')
        if "Result" in game.headers:
            result.append(game.headers["Result"])
        else:
            result.append('*')
        games.append(game.mainline_moves())
        #evals.append(to_eval(game ,20).centipawns)
        end = time.time()
        print(f"time elapsed: {end - start}")
        game = chess.pgn.read_game(file2)

        data = {"White": white, "Black": black, "WhiteELO": white_elo, "BlackELO": black_elo, "Date": date,"Event": event, "Result": result, "Opening": opening, "Rounds": rounds, "Game": games, "Eval": evals, "EventType": event}

        df1 = pd.DataFrame(data)

        df = pd.concat([df,df1])


        df = df.reset_index(drop=True)

        df.to_csv('blitz/' + player +'.csv')

In [30]:
players = ['andreikin, dmitry', 'anand, viswanathan', 'wang, hao', 'grischuk, alexander', 'karjakin, sergey','duda, jan-krzysztof', 'radjabov, teimour', 'dominguez perez, leinier','nakamura, hikaru', 'vachier-lagrave, maxime','aronian, levon','mamedyarov, shakhriyar', 'so, wesley','ding, liren','carlsen, magnus', 'rapport, richard', 'nepomniachtchi, ian', 'giri, anish', 'firouzja, alireza', 'caruana, fabiano','zelcic, robert','khotenashvili, bela', 'bischoff, klaus', 'hoffmann, asa','kaufman, lawrence','bellaiche, elise','vidyarthi, omya', 'eade, don','bueckert, diana','mirza, diana','ryan, john','sloan, sam','szabo, marcell','kiseleva, arina','bhagyashree g patil','kriti mayur patel','wiebke, elfriede','peschk, kathrin','zozulia, anna','marin, irina-luiza','narva, regina','mozelius, peter','eriksson, bo e','hansen, mads','meskenas, vaidas','eidelson, rakhil','kazarian, anna-maja','killer, oliver','israel, delphine','lubbe, melanie','frijling, jeroen','winkels, marcel','smirnova, ekaterina','srebrnic, ana','lucas, marco']
players.reverse()
for player in players:
    print(player)
    pgn_to_csv(player, 600,2)

lucas, marco
srebrnic, ana
1
time elapsed: 0.0
2
time elapsed: 0.0
3
time elapsed: 0.0003249645233154297
4
time elapsed: 0.0
5
time elapsed: 0.0
6
time elapsed: 0.0
7
time elapsed: 0.0
8
time elapsed: 0.0
9
time elapsed: 0.00038123130798339844
10
time elapsed: 0.0
11
time elapsed: 0.0
12
time elapsed: 0.0
13
time elapsed: 0.0002834796905517578
14
time elapsed: 0.0
15
time elapsed: 0.0
16
time elapsed: 0.00028586387634277344
17
time elapsed: 0.0
18
time elapsed: 0.0
19
time elapsed: 0.0
20
time elapsed: 0.0
21
time elapsed: 0.0
22
time elapsed: 0.0
23
time elapsed: 0.000997304916381836
24
time elapsed: 0.0
25
time elapsed: 0.0
26
time elapsed: 0.0
27
time elapsed: 0.0
28
time elapsed: 0.0
29
time elapsed: 0.0
30
time elapsed: 0.0006110668182373047
31
time elapsed: 0.0
32
time elapsed: 0.0009121894836425781
33
time elapsed: 0.0004849433898925781
34
time elapsed: 0.0
35
time elapsed: 0.0
36
time elapsed: 0.0
37
time elapsed: 0.0
38
time elapsed: 0.0004966259002685547
39
time elapsed: 0.00


KeyboardInterrupt



In [ ]:
# data = {"White": [], "Black": [], "WhiteELO": [], "BlackELO": [], "Date": [],"Event": [], "Result": [], "Opening": [], "Rounds": [], "Game": [], "Eval": [], "EventType": []}
#
# df = pd.DataFrame(data)
# for player in players:
#     df.to_csv('blitz/' + player + '.csv')